优质博客：[大江狗](https://pythondjango.cn/python/tools/5-nginx-configuration/)、 [官方文档](https://nginx.org/en/docs/) 、Geekhour

## Configuration File’s Structure
Directives are divided into simple directives and block directives. 简单指令与 块指令。     
If a block directive can have other directives inside braces, it is called a context (examples: events, http, server, and location).

一个配置文件主要包含四个部分：
- 全局块
- events 块
- http 块
- server 块

## 1 全局块
主要⽤来设置⼀些影响Nginx服务器整体运⾏的配置指令，主要包括配置运⾏Nginx服务器的⽤户（组）、允许⽣成的workerprocess数、进程PID存放路径、⽇志存放路径和类型以及配置⽂件引⼊等。

````sh
# 指定运⾏Nginx服务器的⽤户，只能在全局块配置
# 将user指令注释掉，或者配置成nobody的话所有⽤户都可以运⾏
# user [user] [group]
# user nobody nobody;
user nginx;
# 指定⽣成的worker进程的数量，也可使⽤⾃动模式，只能在全局块配置
worker_processes 1;
# 错误⽇志存放路径和类型
error_log /var/log/nginx/error.log warn;
# 进程PID存放路径
pid /var/run/nginx.pid;
````

## 2 events块
````sh
events {
# 指定使⽤哪种⽹络IO模型，只能在events块中进⾏配置
# use epoll
# 每个worker process允许的最⼤连接数
 worker_connections 1024;
}
````

## 3 http块
http块是配置⽂件的主要部分，包括http全局块和server块。
````sh
http {
    # nginx 可以使⽤include指令引⼊其他配置⽂件
    include /etc/nginx/mime.types;
    # 默认类型，如果请求的URL没有包含⽂件类型，会使⽤默认类型
    default_type application/octet-stream; # 默认类型
    # 开启⾼效⽂件传输模式
    sendfile on;
    # 连接超时时间
    keepalive_timeout 65;
    # ......
 ````

## 4 server块
server块是配置虚拟主机的，⼀个http块可以包含多个server块，每个server块就是⼀个虚拟主机。        
#### 区分方式：
- 基于域名：通过 server_name 指令匹配请求的域名（如 example.com）。
- 基于 IP/端口：不同 IP 或端口对应不同虚拟主机。        

比如：当请求 example.com 时，Nginx 匹配第一个 server 块；请求 another.com 时，匹配第二个。      
````sh
http {
    server {
        listen 80;
        server_name example.com;
        root /var/www/example;
        # 其他配置...
    }
    server {
        listen 80;
        server_name another.com;
        root /var/www/another;
        # 其他配置...
    }
}
````

### 4.1 location
location块⽤来配置请求的路由，⼀个server块可以包含多个location块，每个location块就是⼀个请求路由。      

````sh
location [modifier] pattern {
    # 配置指令，如 proxy_pass、root、index 等
}
````
- modifier（可选修饰符）：控制匹配方式。
- pattern：匹配的 URI 模式（如 /api 或 ~ \.php$）

一段反向代理的例子：

````sh
location / {
    # First attempt to serve request as file, then
    # as directory, then fall back to displaying a 404.
    try_files $uri $uri/ =404;
}
location /api/ {
    proxy_pass http://127.0.0.1:8081/;
    proxy_set_header Host $host;
    proxy_set_header X-Real-IP $remote_addr;
    # 支持 SSE 流式输出的关键配置
    proxy_buffering off;
    proxy_cache off;
    proxy_set_header Connection '';
    proxy_http_version 1.1;
    chunked_transfer_encoding on;
}
````